# XPath
<p>XPath 는 'query language'로써 XML의 nodes 들을 선택하는데 용이하다.</p></br>
<p>모든 HTML 문서는 XML 형식으로 작성되므로  webscraping 을 할 때 유용한 문법이다 </p></br>
-https://www.youtube.com/watch?v=btfc_d2rJKk (영어원문 동영상 강의)

-http://nbviewer.jupyter.org/github/TwistedHardware/mltutorial/blob/master/notebooks/quick_tips/4.%20XPath.ipynb (원문)

In [1]:
import requests
from lxml.html import tostring, fromstring, HTMLParser

In [2]:
%%HTML
<html>
  <body>
    <h1>내가 가장 자주쓰는 Python Librarires</h1>
    <ul>
      <li>넘파이 Numpy</li>
      <li>판다스 Pandas</li>
      <li>리퀘스트 requests</li>
    </ul>
  </body>
</html>

## Load HTML Code
cell number 2 에 작성한 `html_code`를 lxml 문법을 활용해서 크롤링 하자

In [3]:
html_code = In[2]
html_code

"get_ipython().run_cell_magic('HTML', '', '<html>\\n  <body>\\n    <h1>내가 가장 자주쓰는 Python Librarires</h1>\\n    <ul>\\n      <li>넘파이 Numpy</li>\\n      <li>판다스 Pandas</li>\\n      <li>리퀘스트 requests</li>\\n    </ul>\\n  </body>\\n</html>')"

In [4]:
# \n 줄바꿈의 특수 문자들이 \\n으로 바뀐걸 \n으로 되돌리기
html_code = html_code[42:-2].replace("\\n","\n")
print(html_code)

<html>
  <body>
    <h1>내가 가장 자주쓰는 Python Librarires</h1>
    <ul>
      <li>넘파이 Numpy</li>
      <li>판다스 Pandas</li>
      <li>리퀘스트 requests</li>
    </ul>
  </body>
</html>


In [5]:
# str ==> Lxml 포맷으로 변환한다.
doc = fromstring(html_code)
doc

<Element html at 0x7f2e60d37278>

# Using xpath to find nodes in a document

## 1. Reading `<h1>` tag
forward slashes / : \`/\`로 문서 root 부터 일치하는 경로의 자료를 크롤링

#### Basic Xpath 사용법
step by step

In [6]:
# inner html만 잘라내기 (/ 루트부터 경로를 지정 to HTML Slicing)
title = doc.xpath("/html/body/h1")[0]
title

<Element h1 at 0x7f2e60d21f98>

In [7]:
# Slicing 결과중 text만 출력
title.text

'내가 가장 자주쓰는 Python Librarires'

In [8]:
title.tag

'h1'

In [9]:
title.attrib

{}

#### Basic Xpath 사용법 2
text() 함수를 Xpath 경로에 포함해서 불러오기

In [10]:
title = doc.xpath("/html/body/h1/text()")[0]
title

'내가 가장 자주쓰는 Python Librarires'

#### 크롤링 가능한 객체가 여럿인 경우
xpath 는 항상 [List 객체]를 출력한다 (위 경우 1개도 list 포맷으로 출력)

문법에 맞는 객체가 없으면 [] 빈 list 객체를 출력한다

In [11]:
# Xpath와 일치하는 객체가 3개 발견되었다.
item_list = doc.xpath("/html/body/ul/li")
item_list

[<Element li at 0x7f2e60d378b8>,
 <Element li at 0x7f2e60d37908>,
 <Element li at 0x7f2e60d37958>]

In [12]:
# 호출 가능한 3개 객체를 `text()` 문법으로 호출하기
doc = fromstring(html_code)
item_list = doc.xpath("/html/body/ul/li/text()")
item_list

['넘파이 Numpy', '판다스 Pandas', '리퀘스트 requests']

## 2. Tag selector 를 활용한 크롤링 (모든 경로를 활용하지 않는다)
Double forward slash `//` : 문서 어디에 있는지를 구분하지 않고, 일치하는 객체를 크롤링 한다 

In [13]:
doc = fromstring(html_code)
item_list = doc.xpath("//li/text()")  # "/html/body/ul/li/text()" 를 생략한 일부분의 입력만으로 출력가능
item_list

['넘파이 Numpy', '판다스 Pandas', '리퀘스트 requests']

#### Selecting one result
/html/body/ul/li/text()

/html/body/ul/li[1]/text()   : [index] index 숫자를 활용하여 해당순번의 객체를 크롤링 

\*\*Notice\*\* : index 번호는 1부터 시작한다!!

In [14]:
doc = fromstring(html_code)
item_list = doc.xpath("/html/body/ul/li[1]/text()")
item_list

['넘파이 Numpy']

# Attributes selector

두개의 `<h1>` tags 에 각기 다른 css 가 정의된 객체들을 구별 가능하다

In [15]:
%%HTML
<html>
  <body>
    <h1 class="text-muted">내가 가장 선호하는 라이브러리 Favorite Python Librarires</h1>
    <ul class="nav nav-pills nav-stacked">
      <li role="presentation"><a href="http://www.numpy.org/">넘파이 Numpy</a></li>
      <li role="presentation"><a href="http://pandas.pydata.org/">판다스 Pandas</a></li>
      <li role="presentation"><a href="http://python-requests.org/">리퀘스트 requests</a></li>
    </ul>
    <h1 class="text-success">Favorite JS Librarires</h1>
    <ul class="nav nav-tabs">
      <li role="presentation"><a href="http://getbootstrap.com/">부트스트랩 Bootstrap</a></li>
      <li role="presentation"><a href="https://jquery.com/">제이쿼리 jQuery</a></li>
      <li role="presentation"><a href="http://d3js.org/">d3.js</a></li>
    </ul>
</html>

In [16]:
html_code = In[15]
html_code = html_code[42:-2].replace("\\n","\n")
print(html_code)

<html>
  <body>
    <h1 class="text-muted">내가 가장 선호하는 라이브러리 Favorite Python Librarires</h1>
    <ul class="nav nav-pills nav-stacked">
      <li role="presentation"><a href="http://www.numpy.org/">넘파이 Numpy</a></li>
      <li role="presentation"><a href="http://pandas.pydata.org/">판다스 Pandas</a></li>
      <li role="presentation"><a href="http://python-requests.org/">리퀘스트 requests</a></li>
    </ul>
    <h1 class="text-success">Favorite JS Librarires</h1>
    <ul class="nav nav-tabs">
      <li role="presentation"><a href="http://getbootstrap.com/">부트스트랩 Bootstrap</a></li>
      <li role="presentation"><a href="https://jquery.com/">제이쿼리 jQuery</a></li>
      <li role="presentation"><a href="http://d3js.org/">d3.js</a></li>
    </ul>
</html>


In [17]:
doc = fromstring(html_code)
doc

<Element html at 0x7f2e60cc0598>

In [18]:
title = doc.xpath("/html/body/h1[@class='text-muted']/text()")[0]
title

'내가 가장 선호하는 라이브러리 Favorite Python Librarires'

## 1. `contains()` 함수 활용하기
Xpath 내부의 [index] 리스트 문법에서, [ index ] 내부에 `contains()`함수를 활용하면 

"full class for selection" 또는 "one of the classed only used"를 사용 가능하다 

In [19]:
item_list = doc.xpath("/html/body/ul[contains(@class,'nav-stacked')]/li/a/text()")
item_list

['넘파이 Numpy', '판다스 Pandas', '리퀘스트 requests']

## 2. Returning attributes

태그 내부의 '속성'값 (ex)`<a>` 태그 `href`속성의 ...link..값) 만 호출가능

/text() 함수 없이도 value만 호출된다

In [20]:
item_list = doc.xpath("/html/body/ul[contains(@class,'nav-stacked')]/li/a/@href")
item_list

['http://www.numpy.org/',
 'http://pandas.pydata.org/',
 'http://python-requests.org/']

# Web Site 에 적용해보기

Read the list of languages with 1M+ articles on http://www.wikipedia.org/

In [21]:
response = requests.get("http://www.wikipedia.org")
doc = fromstring(response.content, parser=HTMLParser(encoding="utf-8"))

In [22]:
lang_list = doc.xpath("//div[@class='langlist langlist-large hlist'][1]/ul/li/a/text()")
lang_list

['Deutsch',
 'English',
 'Español',
 'Français',
 'Italiano',
 'Nederlands',
 '日本語',
 'Polski',
 'Русский',
 'Sinugboanong Binisaya',
 'Svenska',
 'Tiếng Việt',
 'Winaray']